<a href="https://colab.research.google.com/github/yetessam/huggingface/blob/main/notebooks/unit2/smolagents/vision_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision Agents with smolagents


This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

<img src="https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png"  height="100">



## Let's install the dependencies and login to our HF account to access the Inference API

If you haven't installed `smolagents` yet, you can do so by running the following command:

In [1]:
!pip install smolagents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 40.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


Let's also login to the Hugging Face Hub to have access to the Inference API.

In [7]:
from huggingface_hub import notebook_login

notebook_login()

## Providing Images at the Start of the Agent's Execution

In this approach, images are passed to the agent at the start and stored as `task_images` alongside the task prompt. The agent then processes these images throughout its execution.  

Consider the case where Alfred wants to verify the identities of the superheroes attending the party. He already has a dataset of images from previous parties with the names of the guests. Given a new visitor's image, the agent can compare it with the existing dataset and make a decision about letting them in.  

In this case, a guest is trying to enter, and Alfred suspects that this visitor might be The Joker impersonating Wonder Woman. Alfred needs to verify their identity to prevent anyone unwanted from entering.  

Let’s build the example. First, the images are loaded. In this case, we use images from Wikipedia to keep the example minimal, but imagine the possible use-cases!

In [3]:
from PIL import Image
import requests
from io import BytesIO

image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/e/e8/The_Joker_at_Wax_Museum_Plus.jpg",
    "https://upload.wikimedia.org/wikipedia/en/9/98/Joker_%28DC_Comics_character%29.jpg"
]

images = []
for url in image_urls:
    response = requests.get(url)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    images.append(image)

Now that we have the images, the agent will tell us wether the guests is actually a superhero (Wonder Woman) or a villian (The Joker).

In [8]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')



In [23]:
import openai, os

try:
    client = openai.OpenAI()
    response = client.models.list()
    print("Your API key is valid")
    #print("You have access to the following models:")
    #for model in response.data:
    #    print(model.id)

    print(" ")

    completion = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": "write a haiku about ai"}
        ]
    )

    print(completion.choices[0].message.content)

except openai.OpenAIError as e:
    print(f"Error: {e}")
finally:
    client.close()  # Ensure client closure even if an exception occurs






Your API key is valid, and you have access to the following models:
gpt-4.5-preview
gpt-4.5-preview-2025-02-27
gpt-4o-mini-audio-preview-2024-12-17
dall-e-3
dall-e-2
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-realtime-preview
o1-mini-2024-09-12
o1-mini
gpt-4o-mini-audio-preview
whisper-1
gpt-4o-realtime-preview-2024-10-01
babbage-002
chatgpt-4o-latest
tts-1-hd-1106
text-embedding-3-large
gpt-4o-audio-preview-2024-12-17
gpt-4
tts-1-hd
o1-preview
o1-preview-2024-09-12
gpt-4o-2024-08-06
gpt-3.5-turbo-instruct-0914
gpt-4o
tts-1
tts-1-1106
davinci-002
gpt-3.5-turbo-1106
gpt-4-turbo
gpt-3.5-turbo-instruct
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-4o-2024-11-20
gpt-3.5-turbo-0125
gpt-4o-realtime-preview-2024-12-17
gpt-3.5-turbo
gpt-4o-realtime-preview
gpt-3.5-turbo-16k
gpt-4-turbo-2024-04-09
gpt-4o-mini-2024-07-18
gpt-4o-mini
text-embedding-3-small
gpt-4o-2024-05-13
text-embedding-ada-002
gpt-4-1106-preview
gpt-4-0613
omni-moderat

In [24]:
from smolagents import CodeAgent, OpenAIServerModel

model = OpenAIServerModel(model_id="gpt-4o")
#model=HfApiModel(model_id='https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud/')

# Instantiate the agent
agent = CodeAgent(
    tools=[],
    model=model,
    max_steps=20,
    verbosity_level=2
)

from PIL import Image

# Assuming 'images' is your list of images
resized_images = []
for image in images:
    resized_image = image.resize((512, 512))  # Adjust dimensions as needed
    resized_images.append(resized_image)


response = agent.run(
    """
    Describe the costume and makeup that the comic character in these photos is wearing and return the description.
    Tell me if the guest is The Joker or Wonder Woman.
    """,
    images=resized_images
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Describe the costume and makeup that the comic character in these photos is wearing and return the description. │
│     Tell me if the guest is The Joker or Wonder Woman.                                                          │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
The costume and makeup in the photos depict a character with:                                                      
                                                                                                                   
1. **Makeup:**                                                                                                     
   - White facial makeup.                                                                                          
   - Blue eyeshadow around the eyes.                                                                               
   - Red lipstick forming an exaggerated wide smile.                                                               
                                                                                                                   
2. **Costume:**                                                                                                    
   - A purple suit.                                                                                                
   - A yellow shirt or vest.                                                                                       
   - A purple necktie or bow tie.                                                                                  
                                                                                                                   
This character is The Joker.                                                                                       

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
Here is your code snippet:
The costume and makeup in the photos depict a character with:

1. **Makeup:**
   - White facial makeup.
   - Blue eyeshadow around the eyes.
   - Red lipstick forming an exaggerated wide smile.

2. **Costume:**
   - A purple suit.
   - A yellow shirt or vest.
   - A purple necktie or bow tie.

This character is The Joker.
Make sure to include code with the correct pattern, for instance:
Thoughts: Your thoughts
Code:
```py
# Your python code here
```<end_code>
Make sure to provide correct code blobs.

[Step 1: Duration 4.71 seconds| Input tokens: 2,494 | Output tokens: 81]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
I'm unable to identify or analyze people in images directly. However, based on the features you described and the  
typical appearances of the characters:                                                                             
                                                                                                                   
Thought: The character described seems to align with The Joker's typical appearance, characterized by pale skin,   
green hair, and a purple suit.                                                                                     
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
description = {                                                                                                    
    "makeup": "White facial makeup, blue eyeshadow around the eyes, red lipstick in a wide smile.",                
    "costume": "Purple suit, yellow shirt or vest, purple necktie or bow tie."                                     
}                                                                                                                  
character = "The Joker"                                                                                            
print(description, character)                                                                                      
final_answer((description, character))                                                                             
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  description = {                                                                                                  
      "makeup": "White facial makeup, blue eyeshadow around the eyes, red lipstick in a wide smile.",              
      "costume": "Purple suit, yellow shirt or vest, purple necktie or bow tie."                                   
  }                                                                                                                
  character = "The Joker"                                                                                          
  print(description, character)                                                                                    
  final_answer((description, character))                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
{'makeup': 'White facial makeup, blue eyeshadow around the eyes, red lipstick in a wide smile.', 'costume': 'Purple
suit, yellow shirt or vest, purple necktie or bow tie.'} The Joker

Out - Final answer: ({'makeup': 'White facial makeup, blue eyeshadow around the eyes, red lipstick in a wide 
smile.', 'costume': 'Purple suit, yellow shirt or vest, purple necktie or bow tie.'}, 'The Joker')

[Step 3: Duration 8.70 seconds| Input tokens: 9,945 | Output tokens: 326]

In [6]:
response

"Error in generating final LLM output:\n'image'"

In this case, the output reveals that the person is impersonating someone else, so we can prevent The Joker from entering the party!

## Providing Images with Dynamic Retrieval

This examples is provided as a `.py` file since it needs to be run locally since it'll browse the web. Go to the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course) for more details.